In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import os
import random
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output
%matplotlib inline
from chosen_gpu import get_freer_gpu

In [2]:
device = torch.device(get_freer_gpu()) 
print("Configured device: ", device)

Configured device:  cuda:2


In [3]:
par = "par2"
file = "2-Dipesh_2_2021-04-07-07.19.01" 
task = "_visual"
model_name = "cnn"
drift = "drift"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import fnmatch

In [5]:
class Generator(nn.Module):
    """
    Input : random noise / latent vector of any size
    Output : Fake images same size as Real images 
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.activation = nn.LeakyReLU()
        
        self.fc1 = nn.Sequential (nn.Linear(input_size    , hidden_size )  , self.activation, nn.Dropout(0.3) )
        self.fc2 = nn.Sequential (nn.Linear(hidden_size   , hidden_size*2) , self.activation, nn.Dropout(0.3) )
        self.fc3 = nn.Sequential (nn.Linear(hidden_size*2 , hidden_size*4) , self.activation, nn.Dropout(0.3) )
        self.fc4 = nn.Sequential (nn.Linear(hidden_size*4 , output_size )  , self.activation                  )    
    
    def forward(self, X, noise):
        print('GENERATOR')
        X = torch.cat([X, noise], dim = 1)
        print(X.shape)
    
        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.fc4(X)
        X = X.reshape(-1, 3, 28, 28)
        
        return X
    
class Discriminator(nn.Module):
    """
    Input : Real / Fake images
    Output : Classification Real = 1 / Fake = 0
    """
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        
        self.activation = nn.LeakyReLU()
        
        self.fc1 = nn.Sequential (nn.Linear(input_size, hidden_size*6)      , self.activation, nn.Dropout(0.3))
        self.fc2 = nn.Sequential (nn.Linear(hidden_size*6, hidden_size*4)   , self.activation, nn.Dropout(0.3))
        self.fc3 = nn.Sequential (nn.Linear(hidden_size*4, hidden_size*2)   , self.activation  , nn.Dropout(0.3))
        self.fc4 = nn.Sequential (nn.Linear((hidden_size*2)+32, hidden_size), self.activation  , nn.Dropout(0.3))
        
        self.fc5_0 = nn.Sequential (nn.Linear(hidden_size, 32), self.activation  , nn.Dropout(0.3) )  # for real / fake
        self.fc5_1 = nn.Sequential (nn.Linear(32, 16), self.activation  , nn.Dropout(0.3) )
        self.fc5_2 = nn.Sequential (nn.Linear(16, 8),self.activation  , nn.Dropout(0.3)  ) 
        self.fc5_3 = nn.Sequential (nn.Linear(8, 1)  )
        
        self.fc6_0 = nn.Sequential (nn.Linear(hidden_size, 32) , self.activation  , nn.Dropout(0.3)  ) # for number classification
        self.fc6_1 = nn.Sequential (nn.Linear(32, 3) )
    
    def forward(self, X, latent):
        print('DISCRIMINATOR')
               
        X = X.flatten(start_dim = 1)

        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)
        X = torch.cat([X, latent], dim = 1)                          
        X = self.fc4(X)
        
        rf_decision = self.fc5_0(X)
        rf_decision = self.fc5_1(rf_decision)
        rf_decision = self.fc5_2(rf_decision)
        rf_decision = self.fc5_3(rf_decision)
        
        num_decision = self.fc6_0(X)
        num_decision = self.fc6_1(num_decision)
        
        return rf_decision, num_decision